In [6]:
/* SQOOP CODE TO GET THE TABLES


sqoop import --driver com.microsoft.sqlserver.jdbc.SQLServerDriver --connect 'jdbc:sqlserver://bigdata220w18.database.windows.net:1433;database=week3' --username <user> --password "<password>" \
 --query 'SELECT * FROM HH_DEMOGRAPHIC WHERE $CONDITIONS' \
 --split-by household_key -m 1 --target-dir /user/elena/users/hh --fields-terminated-by '\t' --lines-terminated-by '\n' --as-parquetfile
 
sqoop import --driver com.microsoft.sqlserver.jdbc.SQLServerDriver --connect 'jdbc:sqlserver://bigdata220w18.database.windows.net:1433;database=week3' --username <user> --password "<password>" \
 --query 'SELECT * FROM transaction_data WHERE $CONDITIONS' \
 --split-by basket_id -m 1 --target-dir /user/elena/users/trans --fields-terminated-by '\t' --lines-terminated-by '\n' --as-parquetfile
 
sqoop import --driver com.microsoft.sqlserver.jdbc.SQLServerDriver --connect 'jdbc:sqlserver://bigdata220w18.database.windows.net:1433;database=week3' --username <user> --password "<password>" \
 --query 'SELECT * FROM product WHERE $CONDITIONS' \
 --split-by product_id -m 1 --target-dir /user/elena/users/product --fields-terminated-by '\t' --lines-terminated-by '\n' --as-parquetfile
 
 */



hh_demo = [age_desc: string, marital_status_code: string ... 6 more fields]
product = [product_ID: int, manufacturer: int ... 5 more fields]
transaction = [household_key: int, basket_id: bigint ... 10 more fields]


[household_key: int, basket_id: bigint ... 10 more fields]

In [47]:
val hhDF = spark.read.parquet("/user/elena/users/hh/*.parquet").toDF
val prodDF = spark.read.parquet("/user/elena/users/product/*.parquet").toDF
val tranDF = spark.read.parquet("/user/elena/users/trans/*.parquet").toDF

hhDF.createOrReplaceTempView("hhDF")
prodDF.createOrReplaceTempView("prodDF")
tranDF.createOrReplaceTempView("tranDF")

/* create an RFM table for all customers */
val rfm = spark.sql("""SELECT household_key, 
                              MAX(day) as recency, 
                              COUNT(DISTINCT basket_id) as frequency, 
                              SUM(sales_value) as monetary 
                       FROM tranDF 
                       GROUP BY household_key""")

/* add homeownership info to transaction table */
val tran_hh = spark.sql("""SELECT a.*, b.homeowner_desc 
                          FROM tranDF a LEFT JOIN hhDF b 
                          ON a.household_key=b.household_key""")

tran_hh.createOrReplaceTempView("tran_hh")

/* create an RFM table for homeowners */
val rfm_hh = spark.sql("""SELECT household_key, 
                              MAX(day) as recency, 
                              COUNT(DISTINCT basket_id) as frequency, 
                              SUM(sales_value) as monetary 
                       FROM tran_hh 
                       WHERE homeowner_desc = 'Homeowner'
                       GROUP BY household_key
                       """)

rfm.write.csv("file:///data/rfm.csv")
rfm_hh.write.csv("file:///data/rfm_hh.csv")

[Stage 143:===================================================> (195 + 4) / 200]

hhDF = [age_desc: string, marital_status_code: string ... 6 more fields]
prodDF = [product_ID: int, manufacturer: int ... 5 more fields]
tranDF = [household_key: int, basket_id: bigint ... 10 more fields]
rfm = [household_key: int, recency: int ... 2 more fields]
tran_hh = [household_key: int, basket_id: bigint ... 11 more fields]
rfm_hh = [household_key: int, recency: int ... 2 more fields]


lastException: Throwable = null


[household_key: int, recency: int ... 2 more fields]